## import & data

In [409]:
import pandas as pd
import numpy as np
import re
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pandas as pd
import folium
from folium.plugins import HeatMap
from opencage.geocoder import OpenCageGeocode

In [349]:
pd_calls = pd.read_csv("prepared_pd_calls.csv")
pd_calls = pd_calls.drop(pd_calls.columns[0], axis=1)
pd_calls.head()

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,0
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBY AVE
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGE ST
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANO AVE
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWARE ST


In [350]:
pd_calls['street1'] = pd_calls.apply(lambda row: row['Block_Address'].split(' & ')[0] if row['intersection'] else None, axis=1)
pd_calls['street2'] = pd_calls.apply(lambda row: row['Block_Address'].split(' & ')[1] if row['intersection'] and len(row['Block_Address'].split(' & ')) > 1 else None, axis=1)
pd_calls

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street,street1,street2
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,0,SHATTUCK AVE,CHANNING WAY
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBY AVE,None,None
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGE ST,None,None
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANO AVE,None,None
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWARE ST,None,None
...,...,...,...,...,...,...,...,...,...,...
53152,2022-00060808,2022/12/31 00:27:58+00,CFS 415 - Disturbance,Level 4,2000 BERKELEY WAY,False,2000,BERKELEY WAY,None,None
53153,2022-00060803,2022/12/31 00:16:42+00,CFS A911 - Ascertain 911,Level 1,2500 MILVIA ST,False,2500,MILVIA ST,None,None
53154,2022-00060799,2022/12/31 00:15:26+00,CFS A911 - Ascertain 911,Level 1,1800 SOLANO AVE,False,1800,SOLANO AVE,None,None
53155,2023-00090006,2022/12/31 00:13:27+00,CFS 487 - Grand Theft,Level,2600 DWIGHT WAY,False,2600,DWIGHT WAY,None,None


In [352]:
pd_calls["street"].unique()

array(['0', 'ASHBY AVE', 'PAGE ST', 'SOLANO AVE', 'DELAWARE ST', '9TH ST',
       'ALCATRAZ AVE', 'MARTIN LUTHER KING JR WAY', 'FRANCISCO ST',
       'SONOMA AVE', 'TELEGRAPH AVE', 'HEARST AVE', 'UNIVERSITY AVE',
       'RUSSELL ST', 'SAN PABLO AVE', '5TH ST', 'HILGARD AVE',
       'MARINA BLVD', 'SHATTUCK AVE', 'GRANT ST', 'CURTIS ST', '4TH ST',
       'EASTSHORE HWY', 'MARIN AVE', 'DURANT AVE', 'CALIFORNIA ST',
       'BENVENUE AVE', '6TH ST', 'WARD ST', 'KITTREDGE ST',
       'ARLINGTON AVE', 'PARK ST', 'SACRAMENTO ST', 'BOLIVAR DR',
       'HARPER ST', 'ACTON ST', 'CRAGMONT AVE', 'BERKELEY WAY',
       'ALLSTON WAY', 'COLLEGE AVE', 'FAIRVIEW ST', 'ADELINE ST',
       'DWIGHT WAY', 'BONITA AVE', 'SPAULDING AVE', 'DOHR ST',
       'ADDISON ST', 'BANCROFT WAY', 'BONAR ST', '8TH ST', 'PROSPECT ST',
       'CEDAR ST', 'MILVIA ST', 'IDAHO ST', 'WALNUT ST', 'PRINCE ST',
       'HARMON ST', 'NORTHAMPTON AVE', 'EUCLID AVE', 'BLAKE ST',
       'REGAL RD', '7TH ST', 'PARKER ST', 'CRESTON RD',

In [353]:
number_to_word = {
    '9THST': 'NINETHST',
    '4THST': 'FOURTHST',
    '6THST': 'SIXTHST',
    '7THST': 'SEVENTHST',
    '5THST': 'FIFTHST',
    '8THST': 'EIGHTHST',
    '2NDST': 'SECONDST',
    '67THST': 'SIXTYSEVENTHST',
    '10THST': 'TENTHST',
    '66THST': 'SIXTYSIXTHST',
    '63RDST': 'SIXTYTHIRDST',
    '62NDST': 'SIXTYSECONDST'
}

In [354]:
def remove_street(text):
    if text == '0':
        return 'INT'
    if not isinstance(text, str):
        text = str(text)
    text = text.replace(" ", "").upper()  
    
    for number, word in number_to_word.items():
        text = text.replace(number, word)
    text = re.sub(r'[^a-zA-Z]', '', text)
    pattern = re.compile(r'(AVE|ST|CT|RD|WAY|DR|HWY|PL|CIR|CRES)[^A-Za-z]*')
    matches = list(pattern.finditer(text))
    if matches:
        last_match = matches[-1]
        return text[:last_match.end(1)]
    return text

In [355]:
pd_calls['street'] = pd_calls['street'].apply(remove_street)
pd_calls['street1'] = pd_calls['street1'].apply(remove_street)
pd_calls['street2'] = pd_calls['street2'].apply(remove_street)

In [356]:
pd_calls["street"].unique()

array(['INT', 'ASHBYAVE', 'PAGEST', 'SOLANOAVE', 'DELAWAREST', 'NINETHST',
       'ALCATRAZAVE', 'MARTINLUTHERKINGJRWAY', 'FRANCISCOST', 'SONOMAAVE',
       'TELEGRAPHAVE', 'HEARSTAVE', 'UNIVERSITYAVE', 'RUSSELLST',
       'SANPABLOAVE', 'FIFTHST', 'HILGARDAVE', 'MARINABLVD',
       'SHATTUCKAVE', 'GRANTST', 'CURTISST', 'FOURTHST', 'EASTSHOREHWY',
       'MARINAVE', 'DURANTAVE', 'CALIFORNIAST', 'BENVENUEAVE', 'SIXTHST',
       'WARDST', 'KITTREDGEST', 'ARLINGTONAVE', 'PARKST', 'SACRAMENTOST',
       'BOLIVARD', 'HARPERST', 'ACTONST', 'CRAGMONTAVE', 'BERKELEYWAY',
       'ALLSTONWAY', 'COLLEGEAVE', 'FAIRVIEWST', 'ADELINEST', 'DWIGHTWAY',
       'BONITAAVE', 'SPAULDINGAVE', 'DOHRST', 'ADDISONST', 'BANCROFTWAY',
       'BONARST', 'EIGHTHST', 'PROSPECTST', 'CEDARST', 'MILVIAST',
       'IDAHOST', 'WALNUTST', 'PRINCEST', 'HARMONST', 'NORTHAMPTONAVE',
       'EUCLIDAVE', 'BLAKEST', 'REGALRD', 'SEVENTHST', 'PARKERST',
       'CRESTONRD', 'JONESST', 'ELLSWORTHST', 'THEUPL', 'CENTERST',
       

In [357]:
pd_calls["street1"].unique()

array(['SHATTUCKAVE', 'NONE', 'SEVENTHST', 'EIGHTHST', 'ELLSWORTHST',
       'CRAGMONTAVE', 'ENSENADAAVE', 'ACTONST', 'NINETHST',
       'SACRAMENTOST', 'COLLEGEAVE', 'HEARSTAVE', 'ADELINEST', 'TENTHST',
       'REGENTST', 'ALCATRAZAVE', 'DWIGHTWAY', 'ADDISONST',
       'MARTINLUTHERKINGJRWAY', 'CALIFORNIAST', 'GILMANST', 'RIDGERD',
       'BANCROFTWAY', 'ROSEST', 'KALABAGAIWAY', 'VIRGINIAST', 'SOLANOAVE',
       'UNIVERSITYAVE', 'TELEGRAPHAVE', 'FIFTHST', 'OREGONST',
       'THEALAMEDA', 'ARLINGTONAVE', 'CEDARST', 'ASHBYAVE',
       'EASTSHOREHWY', 'SECONDST', 'FOURTHST', 'FAIRVIEWST', 'HARMONST',
       'DURANTAVE', 'HENRYST', 'MURRAYST', 'SANPABLOAVE', 'CENTERST',
       'BERKELEYWAY', 'SANBARBARA', 'MILVIAST', 'TREMONTST', 'BYRONST',
       'CHANNINGWAY', 'SIXTHST', 'COLUSAAVE', 'RUSSELLST',
       'WILDCATCANYONRD', 'LALOMAAVE', 'DELNORTEST', 'PRINCEST', 'VINEST',
       'CORNELLAVE', 'KITTREDGEST', 'ALLSTONWAY', 'KAINSAVE', 'ESSEXST',
       'PAGEST', 'OXFORDST', 'BUENAVISTAWAY',

In [358]:
business_streets = set(business['street'].unique())
filtered_pd_calls = pd_calls[
    (pd_calls['street'].isin(business_streets)) |
     (pd_calls['street1'].isin(business_streets)) |
    (pd_calls['street2'].isin(business_streets))
]
filtered_pd_calls = filtered_pd_calls.reset_index(drop=True)
filtered_pd_calls

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street,street1,street2
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,INT,SHATTUCKAVE,CHANNINGWAY
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBYAVE,NONE,NONE
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGEST,NONE,NONE
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWAREST,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...
48512,2022-00060813,2022/12/31 00:32:18+00,CFS 459 - Burglary,Level 3,2200 DWIGHT WAY,False,2200,DWIGHTWAY,NONE,NONE
48513,2022-00060808,2022/12/31 00:27:58+00,CFS 415 - Disturbance,Level 4,2000 BERKELEY WAY,False,2000,BERKELEYWAY,NONE,NONE
48514,2022-00060803,2022/12/31 00:16:42+00,CFS A911 - Ascertain 911,Level 1,2500 MILVIA ST,False,2500,MILVIAST,NONE,NONE
48515,2022-00060799,2022/12/31 00:15:26+00,CFS A911 - Ascertain 911,Level 1,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE


In [366]:
business = pd.read_csv("business+alcohol.csv")
business = business.drop(business.columns[0], axis=1)
business.head()

,BusDesc,B1_PER_SUB_TYPE,DBA,Business_Location,add,add_num,street,new_name,alcohol
0,CLOTHING STORE,Retail Trade,RADBIRD LLC,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
1,CATERING,Retail Trade,LA CREPE A MOI,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST,LACREPEAMOI,False
2,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
3,MASSAGE THERAPY,Massage,PAPEN GINA,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
4,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False


In [367]:
business["street"].unique()

array(['DOMINGOAVE', 'NINTHST', 'TELEGRAPHAVE', 'PARKERST', 'SANPABLOAVE',
       'ASHBYAVE', 'JOSEPHINEST', 'SHATTUCKAVE', 'DURANTAVE', 'GILMANST',
       'BERKELEY,CA', 'EIGHTHST', 'CENTERST', 'SIXTHST', 'FIFTHST',
       'STUARTST', 'BANCROFTWAY', 'HALCYONCT', 'HOPKINSST', 'FOURTHST',
       'SEVENTHST', 'HILGARDAVE', 'TUNNELRD', 'RIDGERDCDSP', 'EUCLIDAVE',
       'COLLEGEAVEA', 'UNIVERSITYAVE', 'REGENTST', 'PRINCEST',
       'VICENTERD', 'ROOSEVELTAVE', 'HEINZAVE', 'MLKINGJRWAY',
       'ALCATRAZAVE', 'BLAKEST', 'SEVENTHSTB', 'WALNUTSTA',
       'SACRAMENTOST', 'CARLETONST', 'CHANNINGWAY', 'SANPABLOAVEE',
       'ADDISONST', 'BONITAAVE', 'BROOKSIDEDR', 'OXFORDST', 'MCGEEAVE',
       'ALLSTONWAY', 'COLUSAAVE', 'DELAWAREST', 'SIERRAST', 'HEARSTAVE',
       'GRAYSONST', 'CALIFORNIAST', 'ADELINEST', 'OREGONST', 'WALNUTST',
       'LINCOLNST', 'KALABAGAIWAY', 'SHATTUCKAVEC', 'GRANTST',
       'VIRGINIAST', 'SOLANOAVE', 'ACACIAAVE', 'FRANCISCOST',
       'GREENWOODTER', 'FULTONST', 'DERB

In [368]:
def remove_non_alphabetic(text):
    return re.sub(r'[^a-zA-Z]', '', text)

def remove_ca(text):
    if text.endswith('CA'):
        return text[:-2]
    return text

def remove_after_street_abbr(text):
    text = text.upper() 
    pattern = re.compile(r'(AVE|ST|CT|RD|WAY|DR|HWY|PL|CIR|CRES)[^A-Za-z]*')
    matches = list(pattern.finditer(text))
    if matches:
        last_match = matches[-1]
        return text[:last_match.end(1)]
    return text

business['street'] = business['street'].apply(remove_non_alphabetic)
business['street'] = business['street'].apply(remove_ca)
business['street'] = business['street'].apply(remove_after_street_abbr)

In [369]:
business["street"].unique()

array(['DOMINGOAVE', 'NINTHST', 'TELEGRAPHAVE', 'PARKERST', 'SANPABLOAVE',
       'ASHBYAVE', 'JOSEPHINEST', 'SHATTUCKAVE', 'DURANTAVE', 'GILMANST',
       'BERKELEY', 'EIGHTHST', 'CENTERST', 'SIXTHST', 'FIFTHST',
       'STUARTST', 'BANCROFTWAY', 'HALCYONCT', 'HOPKINSST', 'FOURTHST',
       'SEVENTHST', 'HILGARDAVE', 'TUNNELRD', 'RIDGERD', 'EUCLIDAVE',
       'COLLEGEAVE', 'UNIVERSITYAVE', 'REGENTST', 'PRINCEST', 'VICENTERD',
       'ROOSEVELTAVE', 'HEINZAVE', 'MLKINGJRWAY', 'ALCATRAZAVE',
       'BLAKEST', 'WALNUTST', 'SACRAMENTOST', 'CARLETONST', 'CHANNINGWAY',
       'ADDISONST', 'BONITAAVE', 'BROOKSIDEDR', 'OXFORDST', 'MCGEEAVE',
       'ALLSTONWAY', 'COLUSAAVE', 'DELAWAREST', 'SIERRAST', 'HEARSTAVE',
       'GRAYSONST', 'CALIFORNIAST', 'ADELINEST', 'OREGONST', 'LINCOLNST',
       'KALABAGAIWAY', 'GRANTST', 'VIRGINIAST', 'SOLANOAVE', 'ACACIAAVE',
       'FRANCISCOST', 'GREENWOODTER', 'FULTONST', 'DERBYST',
       'VIRGINIAGARD', 'PIEDMONTAVE', 'BURNETTST', 'JONESST', 'TENTHST',
  

In [370]:
business.head()

,BusDesc,B1_PER_SUB_TYPE,DBA,Business_Location,add,add_num,street,new_name,alcohol
0,CLOTHING STORE,Retail Trade,RADBIRD LLC,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
1,CATERING,Retail Trade,LA CREPE A MOI,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST,LACREPEAMOI,False
2,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
3,MASSAGE THERAPY,Massage,PAPEN GINA,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
4,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False


In [374]:
pd_streets = set(pd_calls['street'].unique())
business = business[
    (business['street'].isin(pd_streets))
]
business

,BusDesc,B1_PER_SUB_TYPE,DBA,Business_Location,add,add_num,street,new_name,alcohol
0,CLOTHING STORE,Retail Trade,RADBIRD LLC,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
2,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
3,MASSAGE THERAPY,Massage,PAPEN GINA,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
4,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False
5,SERVICE STATION,Retail Trade,BRIDGEWAY SERVICE INC,"3009 ASHBY AVE\nBERKELEY, CA 94705\n(37.858295...",3009 ASHBY AVE,3009,ASHBYAVE,BRIDGEWAYSERVICE,False
...,...,...,...,...,...,...,...,...,...
8544,RESTAURANT & MARKET,Retail Trade,PIADINA ORIGINAL MARINA LLC,"2905 COLLEGE AVE\nBERKELEY, CA 94705\n(37.8581...",2905 COLLEGE AVE,2905,COLLEGEAVE,PIADINAORIGINALMARINALLC,True
8545,DESIGN & SELL COMPUTER BOARDS,Retail Trade,FURAXA INC,"808 GILMAN ST\nBERKELEY, CA 94710\n(37.8788880...",808 GILMAN ST,808,GILMANST,FURAXA,False
8546,RENTAL PROPERTY,Rental of Real Property,LIAO CHI JUN,"2509 DANA ST\nBERKELEY, CA 94704\n(37.86444599...",2509 DANA ST,2509,DANAST,LIAOCHIJUN,False
8547,CHIROPRACTIC SERVICES TO THE UNDERSERVED,Non Profit Organizations,LOVEJUST COMMUNITY WELLNESS,"2718 TELEGRAPH AVE 210\nBERKELEY, CA 94705\n(3...",2718 TELEGRAPH AVE 210,2718,TELEGRAPHAVE,LOVEJUSTCOMMUNITYWELLNESS,False


In [384]:
street_ranges = business.groupby('street')['add_num'].agg(lambda x: (x.min(), x.max())).reset_index()
street_ranges[['min', 'max']] = pd.DataFrame(street_ranges['add_num'].tolist(), index=street_ranges.index)
street_ranges = street_ranges.drop(columns=['add_num'])

alcohol_count = business[business['alcohol']].groupby('street').size().reset_index(name='alcohol_true_count')

top_busdesc = (business.groupby('street')['BusDesc']
               .agg(lambda x: x.value_counts().head(3).index.tolist())
               .reset_index(name='top_3_busdesc'))

top_b1_per_sub_type = (business.groupby('street')['B1_PER_SUB_TYPE']
                       .agg(lambda x: x.value_counts().head(3).index.tolist())
                       .reset_index(name='top_3_b1_per_sub_type'))

result = pd.merge(street_ranges, alcohol_count, on='street', how='left')
result = pd.merge(result, top_busdesc, on='street', how='left')
result = pd.merge(result, top_b1_per_sub_type, on='street', how='left')

result['alcohol_true_count'] = result['alcohol_true_count'].fillna(0)

result['top_3_busdesc'] = result['top_3_busdesc'].apply(lambda x: ', '.join(x))
result['top_3_b1_per_sub_type'] = result['top_3_b1_per_sub_type'].apply(lambda x: ', '.join(x))
result = result.drop(columns=['min', 'max'])
print(result)

           street  alcohol_true_count  \
0       ACACIAAVE                 0.0   
1       ACTONCRES                 0.0   
2         ACTONST                 0.0   
3           ADAST                 0.0   
4       ADDISONST                 4.0   
..            ...                 ...   
301   WOODMONTAVE                 0.0   
302     WOOLSEYST                 0.0   
303  WPARNASSUSCT                 0.0   
304       YOLOAVE                 0.0   
305    YOSEMITERD                 0.0   

                                         top_3_busdesc  \
0                          CONSULTATION - PSYCHOLOGIST   
1    PROJECT MANAGEMENT, REAL ESTATE DEVELOPMENT, Y...   
2    RESIDENTIAL RENTAL 4 UNITS, RESIDENTIAL RENTAL...   
3    TDM CONSULTING SERVICES, DOCUMENT SHREDDING, A...   
4       COMMERCIAL RENTAL PROPERTY, ATTORNEY, LAW FIRM   
..                                                 ...   
301     ARCHITECTURE & DESIGN, CONSULTANT, HEALTH CARE   
302  RESIDENTIAL RENTAL 3 UNITS, RESIDENTIA

In [414]:
merged_df = pd.merge(filtered_pd_calls, result, on='street', how='left')
merged_df

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street,street1,street2,alcohol_true_count,top_3_busdesc,top_3_b1_per_sub_type
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,INT,SHATTUCKAVE,CHANNINGWAY,NaN,NaN,NaN
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBYAVE,NONE,NONE,4.0,"RENTAL PROPERTY, COMMERCIAL RENTAL PROPERTY, P...","Rental of Real Property, Professional SemiProf..."
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGEST,NONE,NONE,0.0,"ART STUDIO CERAMICS INSTRUCTION, PROPERTY MANA...","Professional SemiProfessional, Rental of Real ..."
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE,20.0,"RESTAURANT, HAIR SALON, COMMERCIAL RENTAL PROP...","Retail Trade, Rental of Real Property, Busines..."
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWAREST,NONE,NONE,0.0,"RENTAL PROPERTY, RESIDENTIAL RENTAL 5 UNITS, R...","Rental of Real Property, Professional SemiProf..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48512,2022-00060813,2022/12/31 00:32:18+00,CFS 459 - Burglary,Level 3,2200 DWIGHT WAY,False,2200,DWIGHTWAY,NONE,NONE,2.0,"RENTAL PROPERTY, RESIDENTIAL RENTAL PROPERTY, ...","Rental of Real Property, Professional SemiProf..."
48513,2022-00060808,2022/12/31 00:27:58+00,CFS 415 - Disturbance,Level 4,2000 BERKELEY WAY,False,2000,BERKELEYWAY,NONE,NONE,0.0,"RENTAL PROPERTY, RENTAL RESIDENTIAL 3 UNITS, C...","Rental of Real Property, Professional SemiProf..."
48514,2022-00060803,2022/12/31 00:16:42+00,CFS A911 - Ascertain 911,Level 1,2500 MILVIA ST,False,2500,MILVIAST,NONE,NONE,2.0,"RESIDENTIAL RENTAL PROPERTY, COMMERCIAL RENTAL...","Rental of Real Property, Professional SemiProf..."
48515,2022-00060799,2022/12/31 00:15:26+00,CFS A911 - Ascertain 911,Level 1,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE,20.0,"RESTAURANT, HAIR SALON, COMMERCIAL RENTAL PROP...","Retail Trade, Rental of Real Property, Busines..."


In [420]:
merged_df['CreateDatetime'] = pd.to_datetime(merged_df['CreateDatetime'])

merged_df['Date'] = merged_df['CreateDatetime'].dt.date
merged_df['Time'] = merged_df['CreateDatetime'].dt.time
merged_df['Hour'] = merged_df['CreateDatetime'].dt.hour
merged_df

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street,street1,street2,alcohol_true_count,top_3_busdesc,top_3_b1_per_sub_type,Date,Time,Hour
0,2023-00062204,2023-12-29 20:27:46+00:00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,INT,SHATTUCKAVE,CHANNINGWAY,NaN,NaN,NaN,2023-12-29,20:27:46,20
1,2023-00062206,2023-12-29 20:26:14+00:00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBYAVE,NONE,NONE,4.0,"RENTAL PROPERTY, COMMERCIAL RENTAL PROPERTY, P...","Rental of Real Property, Professional SemiProf...",2023-12-29,20:26:14,20
2,2023-00062200,2023-12-29 20:12:38+00:00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGEST,NONE,NONE,0.0,"ART STUDIO CERAMICS INSTRUCTION, PROPERTY MANA...","Professional SemiProfessional, Rental of Real ...",2023-12-29,20:12:38,20
3,2023-00062228,2023-12-29 20:05:59+00:00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE,20.0,"RESTAURANT, HAIR SALON, COMMERCIAL RENTAL PROP...","Retail Trade, Rental of Real Property, Busines...",2023-12-29,20:05:59,20
4,2023-00062202,2023-12-29 19:45:10+00:00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWAREST,NONE,NONE,0.0,"RENTAL PROPERTY, RESIDENTIAL RENTAL 5 UNITS, R...","Rental of Real Property, Professional SemiProf...",2023-12-29,19:45:10,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48512,2022-00060813,2022-12-31 00:32:18+00:00,CFS 459 - Burglary,Level 3,2200 DWIGHT WAY,False,2200,DWIGHTWAY,NONE,NONE,2.0,"RENTAL PROPERTY, RESIDENTIAL RENTAL PROPERTY, ...","Rental of Real Property, Professional SemiProf...",2022-12-31,00:32:18,0
48513,2022-00060808,2022-12-31 00:27:58+00:00,CFS 415 - Disturbance,Level 4,2000 BERKELEY WAY,False,2000,BERKELEYWAY,NONE,NONE,0.0,"RENTAL PROPERTY, RENTAL RESIDENTIAL 3 UNITS, C...","Rental of Real Property, Professional SemiProf...",2022-12-31,00:27:58,0
48514,2022-00060803,2022-12-31 00:16:42+00:00,CFS A911 - Ascertain 911,Level 1,2500 MILVIA ST,False,2500,MILVIAST,NONE,NONE,2.0,"RESIDENTIAL RENTAL PROPERTY, COMMERCIAL RENTAL...","Rental of Real Property, Professional SemiProf...",2022-12-31,00:16:42,0
48515,2022-00060799,2022-12-31 00:15:26+00:00,CFS A911 - Ascertain 911,Level 1,1800 SOLANO AVE,False,1800,SOLANOAVE,NONE,NONE,20.0,"RESTAURANT, HAIR SALON, COMMERCIAL RENTAL PROP...","Retail Trade, Rental of Real Property, Busines...",2022-12-31,00:15:26,0


In [422]:
merged_df.to_csv('finalmerge.csv')